In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 29 10:55:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
import json
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import sklearn.preprocessing as preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
from sklearn.preprocessing import StandardScaler

from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD 
from sklearn.decomposition import PCA



#embedding
from gensim.models import Word2Vec

# tqdm
from tqdm.notebook import tqdm



In [40]:
train_file_path = "/content/drive/MyDrive/assign2/comp90051-22-s2-authorship/train.json"
test_file_path = "/content/drive/MyDrive/assign2/comp90051-22-s2-authorship/test.json"

with open(train_file_path, 'r') as f:
    data = json.load(f)
df_train = pd.DataFrame(data)

with open(test_file_path, 'r') as f:
    data = json.load(f)
df_test = pd.DataFrame(data)

In [41]:
# Retrive the non-prolific author and prolific author index
non_prolific_author_index = []
prolific_author_index = []
for i,j in enumerate(df_train['authors']):
  sum = 0
  for k in j:
    if k < 100 and k >= 0:
      sum += 1
  if sum == 0:
    non_prolific_author_index.append(i)
  else:
    prolific_author_index.append(i)

df_non_prolifc_authors = df_train.drop(prolific_author_index)
df_prolific_authors = df_train.drop(non_prolific_author_index)

In [42]:
df_prolific_authors

,authors,year,abstract,venue,title
0,"[42, 13720, 36]",9,"[2455, 1858, 2335, 1543, 1800, 1860, 2000, 286...",20,"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1..."
1,"[1359, 15881, 45]",15,"[40, 1542, 1691, 2449, 1535, 3616, 2206, 1904,...",2,"[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3..."
3,[97],10,"[46, 1624, 1547, 56, 1687, 1644, 6, 7, 3386, 1...",4,"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5..."
4,"[19617, 2]",10,"[37, 3709, 3836, 1586, 2151, 1727, 3021, 1860,...",9,"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,..."
9,"[9641, 44, 5623, 2]",18,"[1731, 2021, 1543, 11, 1546, 11, 1647, 2163, 1...",0,"[1560, 1694, 11, 1546, 11, 3066, 1728, 47, 160..."
...,...,...,...,...,...
25767,"[19974, 68, 8903]",1,"[40, 1542, 1691, 2449, 1535, 2855, 1549, 1755,...",23,"[1708, 33, 58, 57, 2174, 1548, 1538, 1553, 359..."
25776,"[6353, 67, 3037, 15856, 13521]",10,"[40, 1542, 1691, 2907, 1540, 1535, 2121, 2431,...",10,"[53, 2555, 4480, 1535, 1637, 1898, 1549, 53, 3..."
25778,"[7686, 3810, 87, 7451]",16,"[2144, 1661, 2151, 1966, 3776, 4864, 1546, 219...",31,"[1615, 1966, 11, 3238, 1653, 3318, 24, 2144, 1..."
25781,"[14864, 92]",16,"[37, 1662, 33, 1814, 2036, 1692, 1553, 2333, 2...",0,"[46, 1670, 1826, 37, 1632, 46, 1991, 1525, 152..."


In [43]:
# Retrieve the prolific author as label
prolific_author_label = []
non_prolific_author_label = []
for i in df_prolific_authors['authors']:
  prolific_author_label_each_instance = []
  non_prolific_author_label_each_instance = []
  for j in i:
    if j >= 0 and j < 100:
      prolific_author_label_each_instance.append(j)
    else:
      non_prolific_author_label_each_instance.append(j)
  prolific_author_label.append(prolific_author_label_each_instance)
  non_prolific_author_label.append(non_prolific_author_label_each_instance)


df_prolific_authors['coauthors'] = non_prolific_author_label

In [44]:
#One-hot encoding the labels
multilable_binarizer = MultiLabelBinarizer()
ohe_prolific_labels = multilable_binarizer.fit_transform(prolific_author_label).astype('float')

In [45]:
# df_prolific_authors['sentences'] = df_prolific_authors['title'] + df_prolific_authors['abstract'] 
# df_prolific_authors = df_prolific_authors.drop(['abstract','title'],axis = 1)

# df_test['sentences'] = df_test['title'] + df_test['abstract'] 
# df_test = df_test.drop(['abstract','title'],axis = 1)

In [46]:
df_prolific_authors

,authors,year,abstract,venue,title,coauthors
0,"[42, 13720, 36]",9,"[2455, 1858, 2335, 1543, 1800, 1860, 2000, 286...",20,"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1...",[13720]
1,"[1359, 15881, 45]",15,"[40, 1542, 1691, 2449, 1535, 3616, 2206, 1904,...",2,"[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3...","[1359, 15881]"
3,[97],10,"[46, 1624, 1547, 56, 1687, 1644, 6, 7, 3386, 1...",4,"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5...",[]
4,"[19617, 2]",10,"[37, 3709, 3836, 1586, 2151, 1727, 3021, 1860,...",9,"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,...",[19617]
9,"[9641, 44, 5623, 2]",18,"[1731, 2021, 1543, 11, 1546, 11, 1647, 2163, 1...",0,"[1560, 1694, 11, 1546, 11, 3066, 1728, 47, 160...","[9641, 5623]"
...,...,...,...,...,...,...
25767,"[19974, 68, 8903]",1,"[40, 1542, 1691, 2449, 1535, 2855, 1549, 1755,...",23,"[1708, 33, 58, 57, 2174, 1548, 1538, 1553, 359...","[19974, 8903]"
25776,"[6353, 67, 3037, 15856, 13521]",10,"[40, 1542, 1691, 2907, 1540, 1535, 2121, 2431,...",10,"[53, 2555, 4480, 1535, 1637, 1898, 1549, 53, 3...","[6353, 3037, 15856, 13521]"
25778,"[7686, 3810, 87, 7451]",16,"[2144, 1661, 2151, 1966, 3776, 4864, 1546, 219...",31,"[1615, 1966, 11, 3238, 1653, 3318, 24, 2144, 1...","[7686, 3810, 7451]"
25781,"[14864, 92]",16,"[37, 1662, 33, 1814, 2036, 1692, 1553, 2333, 2...",0,"[46, 1670, 1826, 37, 1632, 46, 1991, 1525, 152...",[14864]


In [47]:
def convert_to_string(df):
  total_list = []
  for i in df:
    per_instance = []
    for j in i:
      per_instance.append(str(j))
    total_list.append(per_instance)
  return total_list

# word to vec title model
string_title = convert_to_string(df_train['title'])
# Word2Vec
model_title = Word2Vec(string_title, min_count=1)
print(model_title)

# word to vec abstract model
string_abstract = convert_to_string(df_train['abstract'])
# Word2Vec
model_abstract = Word2Vec(string_abstract, min_count=1)
print(model_abstract)


# word to vec coauthor model
string_coauthor = convert_to_string(df_train['authors'])

model_coauthor = Word2Vec(string_coauthor, min_count=1)
print(model_coauthor)








Word2Vec(vocab=3444, size=100, alpha=0.025)
Word2Vec(vocab=4845, size=100, alpha=0.025)
Word2Vec(vocab=20433, size=100, alpha=0.025)


## title and abstract word to embedding

In [48]:
# # title embedding
# prolific_author_string_title = convert_to_string(df_prolific_authors['title'])
# title_embedding = []
# for i in tqdm(prolific_author_string_title):
#   temp = np.zeros((len(i),100))
#   for k, j in enumerate(i):
#     temp[k,:] = model_title[j]
#   title_embedding.append(temp.mean(axis=0))


# # abstract embedding
# prolific_author_string_abstract = convert_to_string(df_prolific_authors['abstract'])
# abstract_embedding = []
# for i in tqdm(prolific_author_string_abstract):
#   temp = np.zeros((len(i),100))
#   for k, j in enumerate(i):
#     temp[k,:] = model_abstract[j]
#   abstract_embedding.append(temp.mean(axis=0))





# # coauthor embedding
# prolific_author_string_coauthor = convert_to_string(df_prolific_authors['coauthors'])
# coauthor_embedding = []
# for i in tqdm(prolific_author_string_coauthor):
#   temp = np.zeros((len(i),100))
#   for k, j in enumerate(i):
#     temp[k,:] = model_coauthor[j]
#   coauthor_embedding.append(temp.mean(axis=0))








## counter vectorizer

In [49]:
# counts for abstract, title and coauthor
def convert_to_instance_string(df):
  total_list = []
  for i in df:
    total_list.append(str(i)[1:-1].replace(",", ""))
  return total_list

# Counts for abstract
abstract_voc = {}
abstract_num = 4999
for i in range(abstract_num):
  abstract_voc[str(i)] = i




prolific_author_string_abstract = convert_to_instance_string(df_prolific_authors['abstract'])
abstract_vectorizer = CountVectorizer(vocabulary = abstract_voc)
abstract_counts = abstract_vectorizer.fit_transform(prolific_author_string_abstract)
abstract_counts = abstract_counts.toarray()



# Counts for title
title_voc = {}
title_num = 4999
for i in range(title_num):
  title_voc[str(i)] = i




prolific_author_string_title = convert_to_instance_string(df_prolific_authors['title'])
title_vectorizer = CountVectorizer(vocabulary = title_voc)
title_counts = title_vectorizer.fit_transform(prolific_author_string_title)
title_counts = title_counts.toarray()


# Counts for coauthor

coauthor_voc = {}
coauthor_num = 21246
for i in range(coauthor_num):
  coauthor_voc[str(i)] = i

prolific_author_string_coauthors = convert_to_instance_string(df_prolific_authors['coauthors'])
coauthor_vectorizer = CountVectorizer(vocabulary = coauthor_voc)
coauthor_counts = coauthor_vectorizer.fit_transform(prolific_author_string_coauthors)
coauthor_counts = coauthor_counts.toarray()




In [50]:
abstract_counts

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## PCA for counted-based abstract and title

In [51]:


# svd_abstract = TruncatedSVD(n_components=200)
# svd_count_abstract = svd_abstract.fit_transform(abstract_counts)



# svd_title = TruncatedSVD(n_components=100)
# svd_count_title = svd_title.fit_transform(title_counts)



# svd_coauthor = TruncatedSVD(n_components=100)
# svd_count_coauthor = svd_coauthor.fit_transform(coauthor_counts)


pca_200 = PCA(n_components= 200)
pca_count_abstract = pca_200.fit_transform(abstract_counts)

pca_100 = PCA(n_components= 100)
pca_count_title = pca_100.fit_transform(title_counts)


pca_count_coauthor = pca_100.fit_transform(coauthor_counts)


## svd counts abstract and title

In [52]:
# counter vectorized features



coauthors_features_name = ['coauthors_' + str(i) for i in range(100)]
df_coauthors_features = pd.DataFrame(pca_count_coauthor)
df_coauthors_features = df_coauthors_features.astype('float')
df_coauthors_features.columns = coauthors_features_name


abstract_features_name = ['abstract_' + str(i) for i in range(200)]
df_abstract_features = pd.DataFrame(pca_count_abstract)
df_abstract_features = df_abstract_features.astype('float')
df_abstract_features.columns = abstract_features_name

title_features_name = ['title_' + str(i) for i in range(100)]
df_title_features = pd.DataFrame(pca_count_title)
df_title_features = df_title_features.astype('float')
df_title_features.columns = title_features_name


In [53]:
df_merge_coauthors_sentences = pd.concat([df_coauthors_features, df_title_features,df_abstract_features], axis = 1)

In [54]:
df_merge_coauthors_sentences

,coauthors_0,coauthors_1,coauthors_2,coauthors_3,coauthors_4,coauthors_5,coauthors_6,coauthors_7,coauthors_8,coauthors_9,...,abstract_190,abstract_191,abstract_192,abstract_193,abstract_194,abstract_195,abstract_196,abstract_197,abstract_198,abstract_199
0,-0.009571,-0.002846,-0.008844,-0.000511,-0.001075,-0.001923,-0.006986,0.000874,-0.000477,0.000430,...,-0.316160,0.556565,0.194132,-0.410025,-0.349819,-0.085313,0.577743,0.023656,0.123779,0.451074
1,-0.011007,-0.003905,-0.012550,-0.000712,-0.001495,-0.003017,-0.009968,0.001208,-0.000901,0.000935,...,0.124113,0.142072,0.247351,0.190889,0.516250,0.441111,-0.146353,0.278677,-0.005393,0.177249
2,-0.009434,-0.002798,-0.008725,-0.000478,-0.000973,-0.001885,-0.006686,0.000754,-0.000503,0.000484,...,-0.299515,-0.038687,-0.270053,-0.286963,-0.070599,-0.218529,-0.031311,0.447944,-0.078734,0.306917
3,-0.009921,-0.003203,-0.009576,-0.000076,-0.001061,-0.001889,-0.007728,0.000643,-0.001342,-0.000928,...,0.384354,0.188743,0.227883,-0.687872,0.563455,0.179416,0.207539,-0.204147,0.209244,-0.301667
4,-0.010378,-0.004540,-0.012820,-0.000562,-0.001430,-0.003136,-0.010055,0.001154,0.001767,0.001176,...,0.870087,0.495401,-0.001753,-0.538688,-0.053328,-0.016859,0.220541,-1.012202,-0.373231,1.549151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,-0.009283,-0.005068,-0.005012,-0.001051,-0.002071,-0.005167,-0.011075,-0.003089,-0.000469,0.021100,...,0.024630,0.372056,-0.389966,0.100623,-0.315273,0.166802,-0.004482,0.113307,0.450305,-0.196696
7456,-0.009852,-0.003023,-0.009470,-0.000521,-0.001063,-0.002067,-0.007342,0.000830,-0.000558,0.000540,...,0.588100,-0.366446,-0.406382,-0.143317,0.314082,-0.302000,-0.349476,0.570553,0.615917,0.461205
7457,0.092097,0.059339,-0.050990,0.001464,-0.005062,0.027925,-0.003616,0.025550,-0.016458,0.019490,...,1.073664,-0.396448,-0.107901,1.289523,0.272125,-0.314186,-0.580689,-0.066276,0.509617,0.852371
7458,-0.010877,-0.003379,-0.009356,-0.002581,-0.002564,-0.003929,-0.010583,-0.000069,-0.000409,-0.001764,...,-0.213597,-0.007666,0.129398,0.164094,-0.270876,0.020877,-0.232491,-0.106829,0.114766,0.010555


In [55]:
df_coauthors_features

,coauthors_0,coauthors_1,coauthors_2,coauthors_3,coauthors_4,coauthors_5,coauthors_6,coauthors_7,coauthors_8,coauthors_9,...,coauthors_90,coauthors_91,coauthors_92,coauthors_93,coauthors_94,coauthors_95,coauthors_96,coauthors_97,coauthors_98,coauthors_99
0,-0.009571,-0.002846,-0.008844,-0.000511,-0.001075,-0.001923,-0.006986,0.000874,-0.000477,0.000430,...,0.001993,0.000719,0.002881,-0.003325,-0.002331,0.000460,-0.003749,-0.001203,0.000053,-0.001692
1,-0.011007,-0.003905,-0.012550,-0.000712,-0.001495,-0.003017,-0.009968,0.001208,-0.000901,0.000935,...,0.003487,0.008649,0.018916,0.003994,0.000111,0.012183,-0.006647,-0.004866,-0.010021,-0.009317
2,-0.009434,-0.002798,-0.008725,-0.000478,-0.000973,-0.001885,-0.006686,0.000754,-0.000503,0.000484,...,0.000878,0.000633,0.003144,-0.004248,-0.000891,0.001308,-0.004126,-0.001651,-0.000352,-0.001305
3,-0.009921,-0.003203,-0.009576,-0.000076,-0.001061,-0.001889,-0.007728,0.000643,-0.001342,-0.000928,...,-0.002706,0.004429,0.006483,-0.008036,-0.010912,0.010227,-0.005913,0.001391,0.002756,-0.001816
4,-0.010378,-0.004540,-0.012820,-0.000562,-0.001430,-0.003136,-0.010055,0.001154,0.001767,0.001176,...,0.038717,0.008802,0.016410,-0.024823,-0.006347,0.009623,-0.013238,0.009141,0.013797,0.009710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,-0.009283,-0.005068,-0.005012,-0.001051,-0.002071,-0.005167,-0.011075,-0.003089,-0.000469,0.021100,...,-0.098502,-0.012798,-0.086201,0.005014,0.003279,0.018240,0.090990,0.008121,-0.014214,-0.080952
7456,-0.009852,-0.003023,-0.009470,-0.000521,-0.001063,-0.002067,-0.007342,0.000830,-0.000558,0.000540,...,0.001284,0.000818,0.004202,-0.005537,-0.001161,0.001758,-0.005543,-0.002195,-0.000508,-0.001807
7457,0.092097,0.059339,-0.050990,0.001464,-0.005062,0.027925,-0.003616,0.025550,-0.016458,0.019490,...,0.005905,-0.029116,0.002259,-0.006203,-0.007285,-0.001115,0.027806,-0.014079,0.000511,0.004656
7458,-0.010877,-0.003379,-0.009356,-0.002581,-0.002564,-0.003929,-0.010583,-0.000069,-0.000409,-0.001764,...,-0.004989,0.068613,-0.022895,0.067553,-0.016738,0.180067,0.036072,-0.223458,0.000431,0.088796


In [56]:
df_title_features

,title_0,title_1,title_2,title_3,title_4,title_5,title_6,title_7,title_8,title_9,...,title_90,title_91,title_92,title_93,title_94,title_95,title_96,title_97,title_98,title_99
0,-0.104390,-0.899345,-0.434115,0.014167,-0.615432,-0.230758,0.037018,0.429025,-0.064514,0.910843,...,-0.007803,0.216295,-0.038014,-0.054866,-0.202865,0.034769,-0.033622,0.136720,0.134015,-0.048192
1,-0.061744,0.965644,1.560803,0.042176,-0.889819,-0.223741,0.303561,-0.755781,-0.543063,-0.987239,...,0.000317,0.034429,0.035656,0.154158,-0.024986,-0.144253,0.078227,0.176758,0.099190,0.257979
2,-0.715197,-0.552108,0.003134,1.098545,1.593943,-0.659770,0.407286,-0.149940,0.651641,-0.007559,...,0.010791,0.193201,-0.302337,0.069112,0.265473,0.052770,-0.014499,0.155522,-0.192287,-0.184186
3,-0.980859,0.771059,-1.256239,-0.710768,0.153766,0.275984,0.450083,-0.237215,-0.678538,-0.643243,...,0.372838,-0.568613,-0.053789,-0.186024,-0.470164,-0.446689,0.156469,-0.137272,-0.110689,0.130181
4,0.419577,2.150678,1.427501,1.704239,-1.918350,-0.691028,0.416509,0.059435,0.840164,0.076216,...,-0.049131,0.548976,0.360282,-0.155778,-0.228214,-0.087469,0.039023,-0.264784,-0.175169,-0.111621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,-0.952211,-0.586320,-0.080592,0.052563,-0.078157,-0.272398,0.347230,0.132454,-0.063954,0.418303,...,0.049821,0.042997,0.149338,-0.255395,0.079646,0.049009,-0.014125,-0.123616,-0.038850,-0.096674
7456,-0.528040,-0.178056,0.875995,-0.506144,0.287300,1.417883,0.656316,-1.044156,1.382964,0.663548,...,-0.139289,0.145387,-0.303620,-0.313543,-0.021538,0.204740,-0.223923,-0.063029,-0.189510,-0.566919
7457,-0.874029,-0.052301,0.036527,0.618481,-0.904660,-0.119751,-0.011450,-0.180017,-0.040005,-0.597550,...,-0.043845,0.061248,0.168951,0.015619,0.153930,-0.030494,-0.223879,0.186967,0.052016,0.121839
7458,0.145490,-0.656746,-0.272504,0.719806,1.256791,-0.224620,0.453831,-0.242561,0.468189,-0.154747,...,-0.070102,-0.036094,0.051193,0.121419,-0.196160,-0.080483,0.136935,0.059110,0.301246,0.064327


In [57]:
df_abstract_features

,abstract_0,abstract_1,abstract_2,abstract_3,abstract_4,abstract_5,abstract_6,abstract_7,abstract_8,abstract_9,...,abstract_190,abstract_191,abstract_192,abstract_193,abstract_194,abstract_195,abstract_196,abstract_197,abstract_198,abstract_199
0,-6.532645,-1.783645,-2.704608,1.527618,-2.006903,1.253318,1.009644,-2.113763,2.927393,0.692333,...,-0.316160,0.556565,0.194132,-0.410025,-0.349819,-0.085313,0.577743,0.023656,0.123779,0.451074
1,-3.997916,-4.132037,1.799356,-0.724305,2.077230,0.697334,2.564390,0.499511,0.052518,0.757283,...,0.124113,0.142072,0.247351,0.190889,0.516250,0.441111,-0.146353,0.278677,-0.005393,0.177249
2,-4.944162,3.203725,-1.250631,0.836960,-0.399517,0.455560,1.308898,-1.376809,-1.437708,-0.114222,...,-0.299515,-0.038687,-0.270053,-0.286963,-0.070599,-0.218529,-0.031311,0.447944,-0.078734,0.306917
3,-4.402092,-1.638486,-2.341381,1.216474,3.082062,0.124830,0.445660,-0.211135,-0.321364,2.558900,...,0.384354,0.188743,0.227883,-0.687872,0.563455,0.179416,0.207539,-0.204147,0.209244,-0.301667
4,-0.164966,0.380490,7.773817,1.615499,-3.934496,-3.411055,0.852442,0.542791,-0.663889,-1.561843,...,0.870087,0.495401,-0.001753,-0.538688,-0.053328,-0.016859,0.220541,-1.012202,-0.373231,1.549151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,-3.844675,-2.737017,-2.130538,-1.443552,2.562788,0.383540,0.331700,0.347616,0.498375,-0.339795,...,0.024630,0.372056,-0.389966,0.100623,-0.315273,0.166802,-0.004482,0.113307,0.450305,-0.196696
7456,-0.438266,-3.292679,-3.145180,-2.215988,-1.398203,-0.758516,0.452337,-1.710440,-1.307031,0.779634,...,0.588100,-0.366446,-0.406382,-0.143317,0.314082,-0.302000,-0.349476,0.570553,0.615917,0.461205
7457,-0.834171,3.179941,-1.953299,-2.068563,4.067145,0.158689,-0.492177,1.417444,-0.492645,-1.177257,...,1.073664,-0.396448,-0.107901,1.289523,0.272125,-0.314186,-0.580689,-0.066276,0.509617,0.852371
7458,-8.079442,2.027224,-0.992181,0.419949,-0.421532,-1.627133,0.875830,-0.211482,0.129860,1.273333,...,-0.213597,-0.007666,0.129398,0.164094,-0.270876,0.020877,-0.232491,-0.106829,0.114766,0.010555


In [58]:
df_merge_coauthors_sentences.fillna(0,inplace =True)
df_coauthors_features.fillna(0,inplace =True)

In [64]:
X_train,X_test,y_train,y_test = train_test_split(df_merge_coauthors_sentences,ohe_prolific_labels,test_size=0.1,random_state=90051)

## classifierchain model (无深度学习目前最好的model) Logistic Regression

In [60]:
base_lr = LogisticRegression(solver='liblinear', random_state=0,max_iter = 200)
chain = ClassifierChain(base_lr, order='random', random_state=0)
# scaler = preprocessing.StandardScaler().fit(X_train)
# X_scaled_train = scaler.transform(X_train)
# X_scaled_test= scaler.transform(X_test)
chain.fit(X_train, y_train)
predictions = chain.predict(X_test)
f1_score(y_test, predictions, average=None)

array([0.51612903, 0.5       , 0.25      , 0.33333333, 0.46153846,
       0.16666667, 0.23076923, 0.58823529, 0.25      , 0.        ,
       0.07692308, 0.33333333, 0.21052632, 0.22222222, 0.25      ,
       0.55172414, 0.26086957, 0.33333333, 0.5       , 0.5       ,
       0.        , 0.57142857, 0.2       , 0.26666667, 0.13333333,
       0.35294118, 0.48484848, 0.13333333, 0.2       , 0.22222222,
       0.        , 0.4       , 0.34482759, 0.85714286, 0.        ,
       0.15384615, 0.        , 0.4       , 0.34782609, 0.28571429,
       0.30769231, 0.18181818, 0.        , 0.5       , 0.57142857,
       0.71428571, 0.42857143, 0.62068966, 0.        , 0.28571429,
       0.        , 0.4       , 0.        , 0.08      , 0.2       ,
       0.6       , 0.25      , 0.26666667, 0.18181818, 0.60869565,
       0.        , 0.66666667, 0.        , 0.4       , 0.16666667,
       0.11111111, 0.57142857, 0.15384615, 0.28571429, 0.28571429,
       0.55555556, 0.25      , 0.26666667, 0.28571429, 0.45454

In [61]:
f1_score(y_test, predictions, average='samples')

0.24616775836614976

## MLP

In [65]:
# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test= scaler.transform(X_test)

clf = MLPClassifier(random_state=0, max_iter=300, verbose = True,hidden_layer_sizes=(256,128),early_stopping = True)

clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
f1_score(y_test, predictions, average=None)

Iteration 1, loss = 32.13853568
Validation score: 0.000000


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"


Iteration 2, loss = 7.42118243
Validation score: 0.000000


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"


Iteration 3, loss = 6.52791142
Validation score: 0.000000


...

Iteration 49, loss = 0.20019567
Validation score: 0.162202


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"


Iteration 50, loss = 0.18408829
Validation score: 0.166667
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"


array([0.4516129 , 0.        , 0.        , 0.33333333, 0.54545455,
       0.        , 0.42105263, 0.53333333, 0.33333333, 0.25      ,
       0.27586207, 0.30769231, 0.26086957, 0.35294118, 0.57142857,
       0.58333333, 0.26086957, 0.2       , 0.        , 0.66666667,
       0.        , 0.75      , 0.        , 0.28571429, 0.        ,
       0.45454545, 0.48484848, 0.13333333, 0.6       , 0.        ,
       0.25      , 0.28571429, 0.46666667, 0.76190476, 0.2       ,
       0.        , 0.        , 0.70588235, 0.33333333, 0.30769231,
       0.33333333, 0.35294118, 0.        , 0.66666667, 0.57142857,
       0.76923077, 0.42857143, 0.62068966, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.08333333, 0.        ,
       0.83333333, 0.4       , 0.4       , 0.        , 0.6       ,
       0.        , 0.57142857, 0.        , 0.44444444, 0.16666667,
       0.41666667, 0.30769231, 0.30769231, 0.        , 0.16666667,
       0.5       , 0.28571429, 0.16666667, 0.25      , 0.17391

In [66]:
f1_score(y_test, predictions, average='samples')

0.25974084003574616

## MultiOutputClassifier

In [ ]:
base_lr = LogisticRegression(solver='liblinear', random_state=0,max_iter = 200)
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test= scaler.transform(X_test)
clf = MultiOutputClassifier(base_lr).fit(X_scaled_train, y_train)
predictions = clf.predict(X_scaled_test)
f1_score(y_test, predictions, average=None)

array([0.78947368, 0.5       , 0.28571429, 0.22222222, 0.61538462,
       0.28571429, 0.52173913, 0.70588235, 0.33333333, 0.4       ,
       0.375     , 0.58823529, 0.1875    , 0.4       , 0.88888889,
       0.54545455, 0.4       , 0.46153846, 0.4       , 0.66666667,
       0.        , 0.5       , 0.36363636, 0.375     , 0.        ,
       0.43478261, 0.47058824, 0.38095238, 0.26666667, 0.625     ,
       0.        , 0.4       , 0.62068966, 0.8       , 0.44444444,
       0.5       , 0.        , 0.45454545, 0.54545455, 0.23529412,
       0.53333333, 0.33333333, 0.4       , 0.85714286, 0.6       ,
       0.71428571, 0.22222222, 0.70967742, 0.        , 0.5       ,
       0.28571429, 0.28571429, 0.22222222, 0.36842105, 0.        ,
       0.90909091, 0.42857143, 0.73684211, 0.53333333, 0.63157895,
       0.66666667, 0.70588235, 0.2       , 0.66666667, 0.33333333,
       0.56      , 0.57142857, 0.4       , 0.28571429, 0.30769231,
       0.66666667, 0.33333333, 0.52631579, 0.22222222, 0.38461

In [ ]:
f1_score(y_test, predictions, average='samples')

0.40389378271415804

## Test set

In [ ]:
#Test
# Counts for sentences
sentences_voc = {}
sentences_num = 4999
for i in range(sentences_num):
  sentences_voc[str(i)] = i

test_string_sentences = convert_to_string(df_test['sentences'])
test_sentences_vectorizer = CountVectorizer(vocabulary = sentences_voc)
test_sentences_counts = test_sentences_vectorizer.fit_transform(test_string_sentences).toarray()  

# Counts for coauthors
coauthor_voc = {}
coauthor_num = 21246
for i in range(coauthor_num):
  coauthor_voc[str(i)] = i

test_string_coauthor = convert_to_string(df_test['coauthors'])
test_coauthor_vectorizer = CountVectorizer(vocabulary = coauthor_voc)
test_coauthor_counts = coauthor_vectorizer.fit_transform(test_string_coauthor).toarray()



test_coauthors_features_name = ['coauthors_' + str(i) for i in range(21246)]
# coauthors_features = np.vstack([coauthors_features_name,coauthors_ohe])
df_test_coauthors_features = pd.DataFrame(test_coauthor_counts)
df_test_coauthors_features = df_test_coauthors_features.astype('float')
df_test_coauthors_features.columns = test_coauthors_features_name

test_sentence_features_name = ['sentences_' + str(i) for i in range(1,5000)]
df_test_sentence_features = pd.DataFrame(test_sentences_counts)
df_test_sentence_features = df_test_sentence_features.astype('float')
df_test_sentence_features.columns = test_sentence_features_name


df_test_merge_coauthors_sentences = pd.concat([df_test_coauthors_features, df_test_sentence_features], axis = 1)




In [ ]:
df_test_merge_coauthors_sentences

In [ ]:
X_scaled_predict= scaler.transform(df_test_merge_coauthors_sentences)

In [ ]:
predition = chain.predict(X_scaled_predict)

In [ ]:
predition

In [ ]:
predicted_list = [i.tolist() for i in predition]
total_index_list = []
for i in predicted_list:
  index_list = []
  for j,k in enumerate(i):
    if k == 1:
      index_list.append(str(j))
  if len(index_list) != 0:
    total_index_list.append(index_list)
  else:
    total_index_list.append([str(-1)])

predicted_str = []
for i in total_index_list:
  a = ','.join(i)
  predicted_str.append(a)

In [ ]:

df_output = pd.DataFrame({'ID': list(range(800)),'Predict':predicted_str})
df_output.to_csv('predicted_result.csv',index = False)